<a href="https://colab.research.google.com/github/babypotatotang/PolicySuggestion-through-RevitalizationUnusedSchool/blob/main/Data%20Crawling/%20Crawling_StudentsNum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup 
import requests
import re
import os
import sys
import json
from urllib.parse import quote
import urllib.request

In [ ]:
def delete_iframe(post):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 
    (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    content_code =requests .get(post,headers =headers)
    content_soup = BeautifulSoup(content_code .text, 'lxml')
    real_url ="https://blog.naver.com/"+ content_soup .iframe["src"]
    
    return real_url

In [ ]:
def crawling(url):
    url =delete_iframe(url)
    
    content_code =requests .get(url)
    content_soup = BeautifulSoup(content_code .text, 'lxml')
    
    if content_soup .find("div", attrs ={"class":"se-main-container"}):
        text = content_soup .find("div", attrs ={"class":"se-main-container"}).get_text()
    elif content_soup .find("div", attrs ={"id":"postViewArea"}):
        text = content_soup .find("div", attrs ={"id":"postViewArea"}).get_text()
    
    return text

In [ ]:
def call(keyword,display,start):
    #api 키
    client_id ="wGs_lJbJwLwpjr4XNWnz"
    client_secret ="j71ZKDKdEc"
    url ="https://openapi.naver.com/v1/search/blog?query={}&display={}&start={}".format(keyword,display,str(start))
    request = urllib .request .Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib .request .urlopen(request)
    response_body =response .read()
    
    result =response_body .decode('utf-8')
    jsonresult =json .loads(result)
    return jsonresult

In [ ]:
if __name__=="__main__":
    #기본 설정
    keyword =quote("도서관")
    display =100
    
    texts =[] #리스트형
    file =open('content_0to5.txt','a',encoding ='utf-8')
    
    for num in range(0,5):
        count =0
        jsonresult =call(keyword,display, num *9 +1)

        for i in jsonresult['items']:
            url =i['link']
            ck_blog =re .compile('blog.naver.com')
            ck_blog =ck_blog .search(url)
        
            if ck_blog:
                text =crawling(url)
                texts .append(text)
                final_text ="".join(text)
                file .write(final_text)
                count =count + 1

            else:
                print('no more')